## Here I made a very simple decent parser to parse boolean logic expressions

here are the wiki links for the two forms mentioned in the cookbook

[BNF](https://en.wikipedia.org/wiki/Backus%E2%80%93Naur_form)

[EBNF](https://en.wikipedia.org/wiki/Extended_Backus%E2%80%93Naur_form)

In [1]:
from collections import namedtuple

Token = namedtuple('Token', ['type', 'value'])

def token_generator(logic):
    import re

    AND = r'(?P<AND>[Aa][Nn][Dd])'
    OR = r'(?P<OR>[Oo][Rr])'
    NOT = r'(?P<NOT>[Nn][Oo][Tt])'
    VAR = r'(?P<VAR>[a-zA-Z_][a-zA-Z_0-9]*)'
    LP = r'(?P<LP>\()'
    RP = r'(?P<RP>\))'
    WS = r'(?P<WS>\s+)'

    pat = re.compile('|'.join([AND,OR,NOT,VAR,LP,RP,WS]))
    for mo in re.finditer(pat, logic):
        token_type = mo.lastgroup
        token_value = mo.group()
        if token_type != "WS":
            token = Token(token_type, token_value)
            yield token

In [2]:
class LogicParser(object):
    def parse(self, logic_expression):
        self.token_generator = token_generator(logic_expression)
        self.this_token = None
        self.next_token = None
        self.pending_lp = 0
        self._advance()
        print(self.next_token)
        self.expr()
        
    def _advance(self):
        self.this_token, self.next_token = self.next_token, next(self.token_generator, None)
        
    def _accept(self, token_type):
        if self.next_token and self.next_token.type == token_type:
            if token_type == "LP":
                self.pending_lp = self.pending_lp + 1
            elif token_type == "RP":
                self.pending_lp = self.pending_lp - 1
            self._advance()
            return True
        else:
            return False
        
    def _expect(self, token_type):
        if not self._accept(token_type):
            raise SyntaxError('Expected ' + toktype)
            
    def expr(self):
        if self._accept("VAR"):
            print("VAR :" + self.this_token.value)
            self.op()
        elif self._accept("LP"):
            print('(')
            self.pend_lp = self.pending_lp + 1
            self.expr()
            self._accept("RP")
            print(')')
            self.op()
        elif self._accept("NOT"):
            print("NOT")
            self.expr()
        else:
            raise SyntaxError("Expecting a var or ( or NOT")
            
    def op(self):
        if self._accept("AND"):
            print("AND")
            self.expr()
        elif self._accept("OR"):
            print("OR")
            self.expr()
        else:
            if not self.next_token:
                print("End")
            else:
                if self.next_token.type == "RP" and self.pending_lp == 0:
                    raise SyntaxError("No pending left parenthesis for this unexpected right parenthesis!")
                elif self.next_token.type != "RP":
                    raise SyntaxError("There must be AND or OR between vars or expressions!")
            


In [3]:
p = LogicParser()
p.parse("a and b or (not c) and (d) or (a and (not b))")

Token(type='VAR', value='a')
VAR :a
AND
VAR :b
OR
(
NOT
VAR :c
)
AND
(
VAR :d
)
OR
(
VAR :a
AND
(
NOT
VAR :b
)
)
End
